In [3]:
# load style
from ROOT import gROOT

path = '../../'
# path = '/eos/user/y/youyou/'
gROOT.SetMacroPath(path+"atlasstyle/")
gROOT.LoadMacro("AtlasLabels.C")
gROOT.LoadMacro("AtlasStyle.C")
gROOT.LoadMacro("AtlasUtils.C")
from ROOT import SetAtlasStyle, ATLASLabel, myText, myMarkerText
SetAtlasStyle()

Welcome to JupyROOT 6.26/02

Applying ATLAS style settings...



In [225]:
# load lib
import ROOT
import os, sys
import numpy as np
from array import array
import pandas as pd

In [5]:
# load files

# standard model tree
sm_file = path + 'EFT_MINI_TREES/SM.root'
sm_data = ROOT.TFile(sm_file,'READ')
sm_tree = sm_data.vbswy_tree

# int and quad trees
eft_path = path + 'EFT_MINI_TREES/'
files = [(eft_path+i) for i in os.listdir(eft_path)]
files.sort()
files.remove(sm_file)
int_files = [i for i in files if 'INT' in i]
quad_files = [i for i in files if 'QUAD' in i]

# names of all operators we used
operators = [i.split('/')[-1].split('_')[0] for i in int_files]
operators.index('T7')

12

In [6]:
# list of branches we interested in 
names = [g.GetName() for g in sm_tree.GetListOfBranches()]
truth_branch = [i for i in names if 'truth' in i]
truth_branch = list(dict.fromkeys(truth_branch))
truth_branch.sort()

is_reconst = lambda i: not (('truth' in i) or ('weight' in i) or ('pass_baseline' in i))
reconst_branch = [i for i in names if is_reconst(i)]
reconst_branch = list(dict.fromkeys(reconst_branch))
reconst_branch.sort()

reconst_branch[1:]

['gam_e',
 'gam_eta',
 'gam_phi',
 'gam_pt',
 'j0_e',
 'j0_eta',
 'j0_phi',
 'j0_pt',
 'j1_e',
 'j1_eta',
 'j1_phi',
 'j1_pt',
 'jj_dphi_signed',
 'jj_drap',
 'jj_m',
 'jj_pt',
 'lep_e',
 'lep_eta',
 'lep_phi',
 'lep_pt',
 'w_e',
 'w_eta',
 'w_mt',
 'w_phi',
 'w_pt',
 'wy_m',
 'wy_phi',
 'wy_pt']

In [7]:
# def some helper functions
def generate_output_path(operator,add='', format = 'pdf'):
    folder = 'rebin_reconstructed/'
    folder = './output_'+format+'/'+folder
    if not os.path.exists(folder):
        os.mkdir(folder)
    return (folder+operator+add+'.'+format)

generate_output_path('T0')

'./output_pdf/rebin_reconstructed/T0.pdf'

In [8]:
def pre_selection(tree):
    entryList = ROOT.TEntryList("entryListName", "Title of the entry list")
    for entry in tree:
        gam = ROOT.TLorentzVector()
        w_boson = ROOT.TLorentzVector()
        jet0 = ROOT.TLorentzVector()
        jet1 = ROOT.TLorentzVector()
        wy = ROOT.TLorentzVector()

        gam.SetPtEtaPhiE(entry.gam_pt, entry.gam_eta, entry.gam_phi, entry.gam_e)
        w_boson.SetPtEtaPhiE(entry.w_pt, entry.w_eta, entry.w_phi, entry.w_e)
        jet0.SetPtEtaPhiE(entry.j0_pt, entry.j0_eta, entry.j0_phi, entry.j0_e)
        jet1.SetPtEtaPhiE(entry.j1_pt, entry.j1_eta, entry.j1_phi, entry.j1_e)
        wy = gam+w_boson
        # i didn't add it as a branch
        wy_xi = abs(wy.Rapidity()-0.5*(jet0.Rapidity()+jet1.Rapidity())) / entry.jj_drap

        selection = (int(entry.pass_baseline_e) | int(entry.pass_baseline_mu)) and (entry.jj_m > 0.5000) and (entry.jj_drap > 2) and (wy_xi < 0.35)
        if selection:
            entryList.Enter(tree.GetReadEntry())

    tree.SetEntryList(entryList)
    
    df = pd.DataFrame(columns=reconst_branch+['weight'])
    i = 0
    tmp = tree.GetEntryNumber(i)
    while tmp != -1:
        tree.GetEntry(tmp)
        list_tmp = []
        for r in reconst_branch:
            list_tmp += [getattr(tree, r)]
        list_tmp += [tree.weight]
        df.loc[i] = list_tmp
        i += 1
        tmp = tree.GetEntryNumber(i)

    return df   # tree, entryList


In [156]:
def generate_dr_dphi(df):
    df = df.copy()

    def mpi_pi(angle):
        while (angle >= np.pi):
            angle -= 2* np.pi
        while (angle < -1* np.pi):
             angle += 2* np.pi

        return angle

    dr_lepy = []
    dr_wy = []
    dphi_lepy = []
    dphi_wy = []
    for i in range(len(df)):
        gamma = ROOT.TLorentzVector()
        lepton = ROOT.TLorentzVector()
        w_boson = ROOT.TLorentzVector()

        gamma.SetPtEtaPhiE(df['gam_pt'][i],df['gam_eta'][i],df['gam_phi'][i],df['gam_e'][i])
        lepton.SetPtEtaPhiE(df['lep_pt'][i],df['lep_eta'][i],df['lep_phi'][i],df['lep_e'][i])
        w_boson.SetPtEtaPhiE(df['w_pt'][i],df['w_eta'][i],df['w_phi'][i],df['w_e'][i])

        dr_lepy += [lepton.DeltaR(gamma)]
        dr_wy += [w_boson.DeltaR(gamma)]

        if lepton.Rapidity() > gamma.Rapidity():
            lepy_phi_forward = lepton.Phi()
            lepy_phi_backward = gamma.Phi()
        else:
            lepy_phi_forward = gamma.Phi()
            lepy_phi_backward = lepton.Phi()
        dphi_lepy += [mpi_pi(lepy_phi_forward - lepy_phi_backward)]

        if w_boson.Rapidity() > gamma.Rapidity():
            wy_phi_forward = w_boson.Phi()
            wy_phi_backward = gamma.Phi()
        else:
            wy_phi_forward = gamma.Phi()
            wy_phi_backward = w_boson.Phi()
        dphi_wy += [mpi_pi(wy_phi_forward - wy_phi_backward)]

    df['dphi_lepy'] = dphi_lepy
    df['dphi_wy'] = dphi_wy
    df['dr_lepy'] = dr_lepy
    df['dr_wy'] = dr_wy
    
    return df    

In [157]:
sm_df = pre_selection(sm_tree)
sm_df = generate_dr_dphi(sm_df)

sm_df



,dsid,gam_e,gam_eta,gam_phi,gam_pt,j0_e,j0_eta,j0_phi,j0_pt,j1_e,...,w_phi,w_pt,wy_m,wy_phi,wy_pt,weight,dphi_lepy,dphi_wy,dr_lepy,dr_wy
0,363270.0,187.574789,2.042001,-0.503302,47.876445,829.435168,-1.997017,2.031665,221.087766,369.512384,...,-0.263019,107.960786,176.138268,-0.336648,154.881409,0.064699,-0.240210,-0.240283,2.070275,2.070284
1,363270.0,716.515885,-1.210340,2.881138,392.318937,2107.599090,3.068873,0.048355,195.462002,2157.751512,...,-0.059591,51.160642,325.896852,2.851317,342.339108,0.079482,-2.940507,-2.940729,3.138830,3.139038
2,363270.0,76.092459,0.221988,-2.728928,74.255336,409.161278,-2.048432,-0.372453,103.671930,98.808764,...,2.673311,38.194998,46.134245,-3.019382,102.872383,0.058395,0.880951,0.880946,0.894036,0.894031
3,363270.0,28.663095,0.672188,-0.054652,23.217365,1220.408780,-2.114067,2.516062,290.448906,956.535871,...,0.161921,209.252560,48.989156,0.140413,231.981212,0.074781,0.216600,0.216573,0.691692,0.691684
4,363270.0,27.678104,0.501819,2.357176,24.524812,945.942935,3.139888,1.492426,81.739844,336.349470,...,-2.981528,47.384492,73.421696,2.990389,64.877674,0.056811,-0.944477,-0.944481,1.969929,1.969932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19944,363272.0,73.200317,1.333682,-2.003242,36.072645,192.523441,-1.022055,-0.004206,122.415944,363.627920,...,1.695181,41.136863,75.141904,2.762000,21.771143,0.091927,2.584761,2.584763,2.605220,2.605222
19945,363272.0,156.622332,-1.909200,1.346155,45.424844,120.358292,-0.219015,-1.085886,117.030790,898.554284,...,2.376111,32.202403,101.735515,1.765037,67.872401,0.086812,1.030489,1.029957,2.318521,2.318285
19946,363272.0,105.070789,-0.993770,2.689175,68.414883,534.734661,-1.341179,-0.147029,261.314715,491.413472,...,-2.860134,53.229632,81.681235,3.008171,113.677272,0.093809,0.734089,0.733877,1.316264,1.316145
19947,363272.0,28.963492,0.117419,0.791893,28.764971,2425.672927,-1.740736,-2.701662,825.331489,2164.045073,...,0.158732,70.835859,54.745198,0.337810,95.553110,0.039691,-0.633298,-0.633161,1.182595,1.182522


In [158]:
def generate_df(operator):
    i = operators.index(operator)
    int_data = ROOT.TFile(int_files[i],'READ')
    int_tree = int_data.vbswy_tree
    quad_data = ROOT.TFile(quad_files[i], 'READ')
    quad_tree = quad_data.vbswy_tree

    # load values of coupling
    c1, c2, c3 = 1,10,100
    if operator[0] == 'T':
        c1, c2, c3 = 0.1,1,10

    # apply pre selection
    int_df = pre_selection(int_tree)
    quad_df = pre_selection(quad_tree)
    
    int_data.Close()
    quad_data.Close()

    # add drap and dphi
    int_df = generate_dr_dphi(int_df)
    quad_df = generate_dr_dphi(quad_df)

    sm_df['c1_w'] = sm_df['weight']
    sm_df['c2_w'] = sm_df['weight']
    sm_df['c3_w'] = sm_df['weight']
    int_df['c1_w'] = int_df['weight']*c1/2.4
    int_df['c2_w'] = int_df['weight']*c2/2.4
    int_df['c3_w'] = int_df['weight']*c3/2.4
    quad_df['c1_w'] = quad_df['weight']*(c1/2.4)**2
    quad_df['c2_w'] = quad_df['weight']*(c2/2.4)**2
    quad_df['c3_w'] = quad_df['weight']*(c3/2.4)**2

    df = pd.concat([sm_df, int_df, quad_df], keys=["s", "i", "q"])
    return df
    

In [159]:

l1 = ['gam_e','lep_pt','j0_pt','j1_pt','w_mt','wy_m','wy_pt']
l2 = ['jj_m','jj_pt','jj_dphi_signed']
branches = l2+l1

operators_t = [i for i in operators if i[0]=='T']
operators_m = [i for i in operators if i[0]=='M']
dfs_t = [generate_df(i) for i in operators_t]
dfs_m = [generate_df(i) for i in operators_m]


In [163]:
unit_dict= {'gam_e':    "E_{#gamma} [GeV]",
        'lep_pt':       "Lepton p_{T} [GeV/c] ",
        'j0_pt':        "Leading jet p_{T}  [GeV/c] ",
        'j1_pt':        "Second Leading jet p_{T} [GeV/c] ",
        'jj_m':         "m_{jj} [GeV/c^{2}]",
        'jj_pt':        "di-jet p_{T}  [GeV/c] ",
        'jj_dphi_signed': "#Delta#phi_{jj}  [rad] ",
        'w_mt':         "m_{T} (W) [GeV/c^{2}]",
        'wy_m':         "m_{W#gamma} [GeV/c^{2}]",
        'wy_pt':        "W#gamma p_{T}  [GeV/c]",
        'dphi_lepy':    'signed #Delta#phi_{Lepton #gamma}  [rad] ',
        'dphi_wy':      'signed #Delta#phi_{W#gamma}  [rad]',
        'dr_lepy':      '#Delta R_{Lepton #gamma}  [rad]',
        'dr_wy':        '#Delta R_{W#gamma}  [rad]'
        }

unit_dict['lep_pt']

'Lepton p_{T} [GeV/c] '

In [227]:
def plot_contribution(operators, dfs, branches, y_max=0.1, y_min=-0.1, col_width=0.2):

    can = ROOT.TCanvas('can1', 'can1', 3000, 2500)
    can.cd()
    pads = []
    hss = []
    bins = np.linspace(1.5, 1.5, 7).cumprod()
    bins = np.insert(bins, 0, 0)

    #def width
    row_width = 0.192
    left_space = 0.999-row_width*len(dfs)
    bottom_space = 1-col_width*len(branches)

    for i in range(len(dfs)):   #5
        operator = operators[i]
        df = dfs[i].copy()
        df_i = df.loc['i']
        df_q = df.loc['q']

        for j in range(len(branches)):  #4
            branch = branches[j]
            max_x = sm_df[branch].max()
            if max_x > 10:
                bins = [i/bins[-1]*max_x for i in bins]
            else:
                bins = np.linspace(sm_df[branch].min(), max_x, 7)
            bins = array('d',bins)
            can.cd()

            if j!=0 and i!=0:
                pad = ROOT.TPad ("pad"+str(i)+str(j),"pad"+str(i)+str(j) ,j*col_width+bottom_space, i*row_width+left_space, (j+1)*col_width+bottom_space, (i+1)*(row_width)+left_space)
            else:
                if i == 0 and j ==0:
                    pad = ROOT.TPad ("pad"+str(i)+str(j),"pad"+str(i)+str(j) ,j*col_width, i*row_width, (j+1)*col_width+bottom_space, (i+1)*(row_width)+left_space)
                elif i==0:
                    pad = ROOT.TPad ("pad"+str(i)+str(j),"pad"+str(i)+str(j) ,j*col_width+bottom_space, i*row_width, (j+1)*col_width+bottom_space, (i+1)*(row_width)+left_space)
                else:
                    pad = ROOT.TPad ("pad"+str(i)+str(j),"pad"+str(i)+str(j) ,j*col_width, i*row_width+left_space, (j+1)*col_width+bottom_space, (i+1)*(row_width)+left_space)
            pads += [pad]
            pad.Draw()
            pad.cd()

            h_i = ROOT.TH1F("i"+str(i)+str(j), 'hi;'+branch+';numbers    .', len(bins)-1, bins)
            h_q = ROOT.TH1F("q"+str(i)+str(j), 'hq;'+branch+';numbers    .', len(bins)-1, bins)
            h_a = ROOT.TH1F("a"+str(i)+str(j), 'ha;'+branch+';numbers    .', len(bins)-1, bins)
            h_s = ROOT.TH1F("s"+str(i)+str(j), 'hs;'+branch+';numbers    .', len(bins)-1, bins)

            for a in range(len(sm_df)):
                h_s.Fill(sm_df[branch][a], sm_df['weight'][a])
            for a in range(len(df_i)):
                h_i.Fill(df_i[branch][a], df_i['c1_w'][a])
                h_a.Fill(df_i[branch][a], df_i['c1_w'][a])
            for a in range(len(df_q)):
                h_q.Fill(df_q[branch][a], df_q['c1_w'][a])
                h_a.Fill(df_q[branch][a], df_q['c1_w'][a])

            h_i.Divide(h_s)
            h_q.Divide(h_s)
            h_a.Divide(h_s)
            hs = ROOT.THStack (branch+operator+"hs"," THStack ")
            hs.Add (h_i)
            hs.Add (h_q)
            hs.Add (h_a)
            hs.Draw ("NOSTACK, h")

            hs.SetMinimum(y_min)
            hs.SetMaximum(y_max)
            hs.GetYaxis().SetNdivisions (407)
            hs.GetYaxis().SetNdivisions (407)
            hs.GetXaxis().SetTitle(unit_dict[branch]+'   ')
            hs.GetXaxis().SetTitleOffset (0.9)
            hs.GetXaxis().SetTitleSize (0.07)
            hs.GetYaxis().SetTitle('Ratio to SM   ')
            hs.GetYaxis().SetTitleOffset (1.2)
            hs.GetYaxis().SetTitleSize (0.06)

            h_i.SetLineColorAlpha(38, 1)
            h_a.SetLineColorAlpha(46, 0.7)
            h_q.SetLineColorAlpha(ROOT.kOrange, 1)
            h_i.SetMarkerStyle(20)
            h_q.SetMarkerStyle(21)
            h_a.SetMarkerStyle(22)
            h_i.SetMarkerColor(38)
            h_a.SetMarkerColor(46)
            h_q.SetMarkerColor(ROOT.kOrange)
            h_i.SetMarkerSize(3)
            h_a.SetMarkerSize(3)
            h_q.SetMarkerSize(3)

            h1_markers, h2_markers, h3_markers = [], [], []
            for a in range(len(bins)):
                if (h_q.GetBinContent(a) >= y_max):
                    h1_markers += [ROOT.TMarker(h_q.GetBinCenter(a), y_max*0.9,21)]
                if (h_a.GetBinContent(a) >= y_max):
                    h2_markers += [ROOT.TMarker(h_a.GetBinCenter(a), y_max*0.9,22)]
                if (h_i.GetBinContent(a) <= y_min):
                    h3_markers += [ROOT.TMarker(h_a.GetBinCenter(a), y_min*0.9,22)]
            for b in h1_markers:
                b.SetMarkerSize(2)
                b.SetMarkerColor(ROOT.kOrange)
                b.Draw('same')
            for b in h2_markers:
                b.SetMarkerSize(2)
                b.SetMarkerColor(46)
                b.Draw('same')
            for b in h3_markers:
                b.SetMarkerSize(2)
                b.SetMarkerColor(38)
                b.Draw('same')
            hss += [h1_markers, h2_markers, h3_markers]

            t = ROOT.TLatex()
            t.SetNDC()
            t.SetTextColor( 1 )
            t.SetTextSize( 0.13 )
            t.SetTextAlign( 12 )
            if operator == 'M2':
                t.SetTextSize( 0.11 )
            t.DrawLatex( 0.85, 0.85, '#scale[1]{'+operator+'}')

            hss += [h_i, h_q, h_a, hs]
            pad.RedrawAxis('g')
            pad.SetBottomMargin (0)
            pad.SetTopMargin (0)
            pad.SetRightMargin(0)
            pad.SetLeftMargin(0)
            if j == 0:
                pad.SetLeftMargin(0.18)
                legend = ROOT.TLegend(0.25,0.65,0.6,0.97)
                c=0.1
                if operator[0]=='M':
                    c=1
                legend.SetHeader('f_{0} /#Lambda^{3} = {1} TeV^{2}'.format({operator}, c,{-4},{4}),"lp")
                legend.AddEntry ( h_i ,'O_{0}'.format({'Interference term'}),"lp")
                legend.AddEntry ( h_q ,'O_{0}'.format({'Quadratic term'}),"lp")
                legend.AddEntry ( h_a ,'O_{0}'.format({'Total aQGC contribution'}),"lp")
                legend.Draw (" same ")
                legend.SetFillColorAlpha(0, 0.8)
                hss += [legend]
            if i == 0:
                hs.GetYaxis().SetTitleSize (0.05)
                hs.GetYaxis().SetTitleOffset (1.35)
                hss+=[hs]
                pad.SetBottomMargin (0.16)
    
    can.Draw()
    can.Update()
    can.Print(branches[0]+'test.pdf') 




In [228]:
os = ['M2', 'T6', 'T1', 'T5', 'T0']
df_arr = [dfs_m[2], dfs_t[4], dfs_t[1], dfs_t[3], dfs_t[0]]
bs1 = ['lep_pt','gam_e','wy_m', 'jj_pt']
bs2 =['jj_m','jj_dphi_signed','dphi_lepy','dr_lepy']

plot_contribution(os,df_arr,bs1,y_max=0.5, y_min=-0.15, col_width=0.24)
plot_contribution(os,df_arr,bs2,y_max=0.005, y_min=-0.005, col_width=0.24)

Warning in <TROOT::Append>: Replacing existing TH1: i00 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: q00 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: a00 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: i01 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: q01 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: a01 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: i02 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: q02 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: a02 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: i03 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: q03 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: a03 (Potential memory leak).
Warning in <TROOT::Append>: 